In [ ]:
pip install holidays

In [11]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import holidays
import time

from Help_Funs import smape, is_holiday

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import tensorflow as tf


s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/test.csv'
file_key_3 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/sample_submission.csv'
file_key_4 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/TPSSEP22_GDP_data_2017_to_2021.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
train['date'] = pd.to_datetime(train['date'], format = '%Y-%m-%d')

test = pd.read_csv(file_content_stream_2)
test['date'] = pd.to_datetime(test['date'], format = '%Y-%m-%d')

submission = pd.read_csv(file_content_stream_3)
country_gdp = pd.read_csv(file_content_stream_4)

important_dates = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 124, 125, 126, 127, 140, 141, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 
                   180, 181, 203, 230, 231, 232, 233, 234, 282, 289, 290, 307, 308, 309, 310, 311, 312, 313, 317, 318, 319, 320, 360, 361, 362, 363, 364, 365]
    
## Basic feature engineering 
train['weekday'] = train['date'].dt.dayofweek
train['month'] = train['date'].dt.month
train['weekend'] = np.where(train['weekday'] >= 5, 1, 0)
train['dayOfMonth'] = train['date'].dt.day
train['dayOfYear'] = train['date'].dt.dayofyear
train['quarter'] = train['date'].dt.quarter
train['year'] = train['date'].dt.year
train['month_sin'] = np.sin(train['month'] * (2 * np.pi / 12))
# train['month_cos'] = np.cos(train['month'] * (2 * np.pi / 12))
train['day_sin'] = np.sin(train['dayOfMonth'] * (2 * np.pi / 12))
# train['day_cos'] = np.cos(train['dayOfMonth'] * (2 * np.pi / 12))
# train['important_dates'] = train['dayOfYear'].apply(lambda x: x if x in important_dates else 0)

test['weekday'] = test['date'].dt.dayofweek
test['month'] = test['date'].dt.month
test['weekend'] = np.where(test['weekday'] >= 5, 1, 0)
test['dayOfMonth'] = test['date'].dt.day
test['dayOfYear'] = test['date'].dt.dayofyear
test['quarter'] = test['date'].dt.quarter
test['year'] = test['date'].dt.year
test['month_sin'] = np.sin(test['month'] * (2 * np.pi / 12))
# test['month_cos'] = np.cos(test['month'] * (2 * np.pi / 12))
test['day_sin'] = np.sin(test['dayOfMonth'] * (2 * np.pi / 12))
# test['day_cos'] = np.cos(test['dayOfMonth'] * (2 * np.pi / 12))
# test['important_dates'] = test['dayOfYear'].apply(lambda x: x if x in important_dates else 0)

## Appending GDP
train = pd.merge(train, country_gdp, on = ['country', 'year'], how = 'left')
train = train.drop(columns = ['year'], axis = 1)

test = pd.merge(test, country_gdp, on = ['country', 'year'], how = 'left')
test = test.drop(columns = ['year'], axis = 1)

## Appending holidays
data_holidays = is_holiday(train, test)
train = data_holidays[0]
test = data_holidays[1]

/home/ec2-user/SageMaker/Analytics_Data_Science/Tabular-Playground-Series/Tabular-Playground-Sep-2022/Help_Funs.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_temp['is_holiday'][j] = np.where(train_temp['date'][j] in holiday_to_use, 1, 0)
/home/ec2-user/SageMaker/Analytics_Data_Science/Tabular-Playground-Series/Tabular-Playground-Sep-2022/Help_Funs.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_temp['black_friday_cyber_monday'][j] = 0
/home/ec2-user/SageMaker/Analytics_Data_Science/Tabular-Playground-Series/Tabular-Playground-Sep-2022/Help_Funs.py:109: SettingWithCopyWarning: 
A value is trying 

In [12]:
train[train['date'] == pd.to_datetime('2017-05-14')].reset_index(drop = True)

,row_id,date,country,store,product,num_sold,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,GDP,is_holiday,holiday_season,black_friday_cyber_monday
0,6384,2017-05-14,Belgium,KaggleMart,Kaggle Advanced Techniques,505,6,5,1,14,134,2,0.5,0.866025,0.5015,0,0,0
1,6385,2017-05-14,Belgium,KaggleMart,Kaggle Getting Started,292,6,5,1,14,134,2,0.5,0.866025,0.5015,0,0,0
2,6386,2017-05-14,Belgium,KaggleMart,Kaggle Recipe Book,249,6,5,1,14,134,2,0.5,0.866025,0.5015,0,0,0
3,6387,2017-05-14,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,460,6,5,1,14,134,2,0.5,0.866025,0.5015,0,0,0
4,6388,2017-05-14,Belgium,KaggleRama,Kaggle Advanced Techniques,184,6,5,1,14,134,2,0.5,0.866025,0.5015,0,0,0
5,6389,2017-05-14,Belgium,KaggleRama,Kaggle Getting Started,106,6,5,1,14,134,2,0.5,0.866025,0.5015,0,0,0
6,6390,2017-05-14,Belgium,KaggleRama,Kaggle Recipe Book,90,6,5,1,14,134,2,0.5,0.866025,0.5015,0,0,0
7,6391,2017-05-14,Belgium,KaggleRama,Kaggle for Kids: One Smart Goose,165,6,5,1,14,134,2,0.5,0.866025,0.5015,0,0,0
8,6392,2017-05-14,France,KaggleMart,Kaggle Advanced Techniques,531,6,5,1,14,134,2,0.5,0.866025,2.5890,0,0,0
9,6393,2017-05-14,France,KaggleMart,Kaggle Getting Started,306,6,5,1,14,134,2,0.5,0.866025,2.5890,0,0,0


In [3]:
train.describe()

,row_id,num_sold,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,GDP,is_holiday,holiday_season,black_friday_cyber_monday
count,70128.000000,70128.000000,70128.000000,70128.000000,70128.000000,70128.000000,70128.000000,70128.000000,7.012800e+04,7.012800e+04,70128.000000,70128.000000,70128.000000,70128.000000
mean,35063.500000,194.296986,2.997947,6.522930,0.285421,15.729637,183.125257,2.508556,-4.784655e-03,1.120044e-01,1.822412,0.031257,0.084873,0.013689
std,20244.354176,126.893874,2.000526,3.448728,0.451618,8.800155,105.439394,1.117091,7.057634e-01,6.960984e-01,1.172973,0.174013,0.278695,0.116198
min,0.000000,19.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,-1.000000e+00,-1.000000e+00,0.501500,0.000000,0.000000,0.000000
25%,17531.750000,95.000000,1.000000,4.000000,0.000000,8.000000,92.000000,2.000000,-8.660254e-01,-5.000000e-01,0.594260,0.000000,0.000000,0.000000
50%,35063.500000,148.000000,3.000000,7.000000,0.000000,16.000000,183.000000,3.000000,-2.449294e-16,2.388680e-15,1.653500,0.000000,0.000000,0.000000
75%,52595.250000,283.000000,5.000000,10.000000,1.000000,23.000000,274.000000,4.000000,5.000000e-01,8.660254e-01,2.603000,0.000000,0.000000,0.000000
max,70127.000000,986.000000,6.000000,12.000000,1.000000,31.000000,366.000000,4.000000,1.000000e+00,1.000000e+00,3.962000,1.000000,1.000000,1.000000


In [ ]:
train[train['date'] == pd.to_datetime('2018-11-22')].reset_index(drop = True)

In [2]:
train.head(10)

,row_id,date,country,store,product,num_sold,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,GDP,is_holiday,holiday_season,black_friday_cyber_monday
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663,6,1,1,1,1,1,0.5,0.500000,0.5015,1,0,0
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615,6,1,1,1,1,1,0.5,0.500000,0.5015,1,0,0
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480,6,1,1,1,1,1,0.5,0.500000,0.5015,1,0,0
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710,6,1,1,1,1,1,0.5,0.500000,0.5015,1,0,0
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240,6,1,1,1,1,1,0.5,0.500000,0.5015,1,0,0
5,5,2017-01-01,Belgium,KaggleRama,Kaggle Getting Started,187,6,1,1,1,1,1,0.5,0.500000,0.5015,1,0,0
6,6,2017-01-01,Belgium,KaggleRama,Kaggle Recipe Book,158,6,1,1,1,1,1,0.5,0.500000,0.5015,1,0,0
7,7,2017-01-01,Belgium,KaggleRama,Kaggle for Kids: One Smart Goose,267,6,1,1,1,1,1,0.5,0.500000,0.5015,1,0,0
8,48,2017-01-02,Belgium,KaggleMart,Kaggle Advanced Techniques,514,0,1,0,2,2,1,0.5,0.866025,0.5015,0,0,0
9,49,2017-01-02,Belgium,KaggleMart,Kaggle Getting Started,408,0,1,0,2,2,1,0.5,0.866025,0.5015,0,0,0


In [4]:
all_data = pd.concat([train, test], axis = 0)

le = LabelEncoder()
cols = ['country', 'store', 'product']
for col in cols:
    le = LabelEncoder()
    all_data[col] = le.fit_transform(all_data[col])

scaler = MinMaxScaler()    

all_data = all_data.drop(['date', 'row_id'], axis = 1)
train = all_data.iloc[:70128,:]
test = all_data.iloc[70128:,:].drop(['num_sold'], axis = 1)
test = pd.DataFrame(scaler.fit_transform(test), columns = test.columns)

X = train.drop(['num_sold'], axis = 1)
Y = train['num_sold']

In [5]:
X.head()

,country,store,product,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,GDP,is_holiday,holiday_season,black_friday_cyber_monday
0,0,0,0,6,1,1,1,1,1,0.5,0.5,0.5015,1,0,0
1,0,0,1,6,1,1,1,1,1,0.5,0.5,0.5015,1,0,0
2,0,0,2,6,1,1,1,1,1,0.5,0.5,0.5015,1,0,0
3,0,0,3,6,1,1,1,1,1,0.5,0.5,0.5015,1,0,0
4,0,1,0,6,1,1,1,1,1,0.5,0.5,0.5015,1,0,0


In [6]:
t1 = time.time()
kf = KFold(n_splits = 4, shuffle = True, random_state = 888)
score_list_tf = []
test_preds_tf = []
fold = 1

## Defining model 
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(16, input_dim = 15, activation = 'relu'),
        tf.keras.layers.Dense(16, activation = 'relu'),
        tf.keras.layers.Dense(1)
])

model.compile(optimizer = 'adam', loss = 'mean_absolute_error')


for train_index, test_index in kf.split(X, Y):
    
    ## Splitting the data
    X_train , X_val = X.iloc[train_index], X.iloc[test_index]  
    Y_train, Y_val = Y.iloc[train_index], Y.iloc[test_index]    
    
    print("X_train shape is :", X_train.shape, "X_val shape is", X_val.shape)
    
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
    X_val = pd.DataFrame(scaler.fit_transform(X_val), columns = X_val.columns)

    model.fit(X_train, Y_train, verbose = 1, epochs = 20, batch_size = 32, validation_data = (X_val, Y_val))
    result = pd.DataFrame(model.predict(X_val))    
    result.columns = ['pred_num_sold']
    result['pred_num_sold'] = [0 if i <= 0 else i for i in result['pred_num_sold']]
    
    Y_val = pd.DataFrame(Y_val.reset_index(drop = True))
    score = smape(Y_val['num_sold'], result['pred_num_sold'])
    print('Fold ', str(fold), ' result is:', score, '\n')
    score_list_tf.append(score)

    test_preds_tf.append(model.predict(test))
    fold += 1

t2 = time.time()
print('TF model with cross validation take : {:.3f} sn.'.format(t2-t1))

2022-09-20 10:37:19.863808: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-20 10:37:19.863872: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-16-161-167.ec2.internal): /proc/driver/nvidia/version does not exist
2022-09-20 10:37:19.865959: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


X_train shape is : (52596, 15) X_val shape is (17532, 15)
Epoch 1/20
1644/1644 [==============================] - 3s 1ms/step - loss: 100.1099 - val_loss: 61.5855
Epoch 2/20
1644/1644 [==============================] - 2s 2ms/step - loss: 56.4184 - val_loss: 53.7183
Epoch 3/20
1644/1644 [==============================] - 3s 2ms/step - loss: 53.1064 - val_loss: 52.1495
Epoch 4/20
1644/1644 [==============================] - 2s 1ms/step - loss: 51.5348 - val_loss: 50.6744
Epoch 5/20
1644/1644 [==============================] - 2s 1ms/step - loss: 50.2035 - val_loss: 49.2358
Epoch 6/20
1644/1644 [==============================] - 2s 1ms/step - loss: 48.7770 - val_loss: 47.4789
Epoch 7/20
1644/1644 [==============================] - 2s 1ms/step - loss: 46.2328 - val_loss: 44.5945
Epoch 8/20
1644/1644 [==============================] - 2s 2ms/step - loss: 43.2605 - val_loss: 41.7453
Epoch 9/20
1644/1644 [==============================] - 2s 1ms/step - loss: 41.3605 - val_loss: 40.0845
Epoch

In [7]:
mean = sum(score_list_tf) / len(score_list_tf)
variance = sum([((x - mean) ** 2) for x in score_list_tf]) / len(score_list_tf)
res = variance ** 0.5
print("Cross validation mean score:", sum(score_list_tf) / len(score_list_tf))
print("Cross validation score's Standart deviation is:", res)

Cross validation mean score: 14.416123551988985
Cross validation score's Standart deviation is: 1.5575080587718084


In [8]:
test_preds_tf = pd.DataFrame(np.concatenate(test_preds_tf, axis = 1))
print(test_preds_tf.shape)

test_preds_tf = test_preds_tf.mean(axis = 1)
print(test_preds_tf.head())

(17520, 4)
0    441.710205
1    294.507202
2    248.570679
3    421.325439
4    160.128937
dtype: float32


In [9]:
submission['num_sold'] = test_preds_tf
submission.to_csv('TF_submission_10.csv', index = False)
submission.head()

,row_id,num_sold
0,70128,441.710205
1,70129,294.507202
2,70130,248.570679
3,70131,421.325439
4,70132,160.128937
